## 이 노드의 루브릭       

1. pix2pix 모델 학습을 위해 필요한 데이터셋을 적절히 구축하였다.       
: 데이터 분석 과정 및 한 가지 이상의 augmentation을 포함한 데이터셋 구축 과정이 체계적으로 제시되었다.   


2. pix2pix 모델을 구현하여 성공적으로 학습 과정을 진행하였다.      
: U-Net generator, discriminator 모델 구현이 완료되어 train_step이 안정적으로 진행됨을 확인하였다.         


3. 학습 과정 및 테스트에 대한 시각화 결과를 제출하였다.             
: 10Epoch 이상의 학습을 진행한 후 최종 테스트 결과에서 진행한 epoch 수에 걸맞는 품질을 확인하였다.